In [1]:
# 라이브러리
import os
import glob
import zlib
import struct
import olefile
import re

In [2]:
def parse_hwp_all(file_path: str) -> str:
    """HWP 파일에서 전체 텍스트를 추출합니다. (BodyText 파싱)"""
    if not os.path.exists(file_path):
        print(f"'{file_path}' 파일을 찾을 수 없습니다.")
        return ""
    try:
        f = olefile.OleFileIO(file_path)
        
        # 압축 여부 확인
        header = f.openstream("FileHeader").read()
        is_compressed = header[36] & 1
        
        texts = []
        
        # 모든 BodyText 섹션 처리
        for entry in f.listdir():
            if entry[0] == "BodyText":
                data = f.openstream(entry).read()
                
                # 압축 해제
                if is_compressed:
                    try:
                        data = zlib.decompress(data, -15)
                    except:
                        pass
                
                # 레코드 파싱
                i = 0
                while i < len(data):
                    if i + 4 > len(data):
                        break
                    
                    rec_header = struct.unpack_from('<I', data, i)[0]
                    rec_type = rec_header & 0x3FF
                    rec_len = (rec_header >> 20) & 0xFFF
                    
                    # 확장 길이 처리
                    if rec_len == 0xFFF:
                        if i + 8 > len(data):
                            break
                        rec_len = struct.unpack_from('<I', data, i + 4)[0]
                        i += 8
                    else:
                        i += 4
                    
                    if i + rec_len > len(data):
                        break
                    
                    # HWPTAG_PARA_TEXT (67)
                    if rec_type == 67 and rec_len > 0:
                        text_data = data[i:i + rec_len]
                        try:
                            text = text_data.decode('utf-16le', errors='ignore')
                            # HWP 제어문자 제거 (0x00~0x1F 범위의 특수 제어코드)
                            cleaned = []
                            for char in text:
                                code = ord(char)
                                if code >= 32 or char in '\n\r\t':
                                    cleaned.append(char)
                                elif code in [13, 10]:  # CR, LF
                                    cleaned.append('\n')
                            text = ''.join(cleaned).strip()
                            if text:
                                texts.append(text)
                        except:
                            pass
                    
                    i += rec_len
        
        f.close()
        
        if texts:
            result = '\n'.join(texts)
            
            # 1. 깨진 문자 제거 (한글, 영문, 숫자, 공백, 로마숫자, 기본 문장부호 유지)
            result = re.sub(r'[^\uAC00-\uD7A3\u2160-\u217Fa-zA-Z0-9\s.,!?():\-\[\]<>~·%/@\'"_=+○●■□▶◆※]', '', result)
            
            # 2. 연속 공백/줄바꿈 정리
            result = re.sub(r'\n{3,}', '\n\n', result)
            result = re.sub(r' {2,}', ' ', result)
            
            print(f"'{file_path}' 파일 파싱 성공!")
            return result.strip()
        else:
            print(f"'{file_path}' 텍스트 추출 실패")
            return ""
            
    except Exception as e:
        print(f"'{file_path}' 파일 파싱 중 오류 발생: {e}")
        return ""

In [3]:
# data 폴더의 모든 HWP 파일 파싱
hwp_files = glob.glob("data/*.hwp")
print(f"발견된 HWP 파일: {len(hwp_files)}개\n")

parsed_docs = {}
for file_path in hwp_files:
    file_name = os.path.basename(file_path)
    print(f"--- {file_name} ---")
    text = parse_hwp_all(file_path)
    if text:
        parsed_docs[file_name] = text
        print(f"[추출된 텍스트 미리보기]")
        print(text[:500])
        print("...\n")
        
        # txt 파일로 저장 (기존 파일 있으면 삭제 후 새로 생성)
        output_file = f"data/{file_name.replace('.hwp', '_parsed.txt')}"
        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"기존 파일 삭제: {output_file}")
            
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"저장 완료: {output_file} ({len(text)}자)\n")
    else:
        print("텍스트 추출 실패\n")

print(f"총 {len(parsed_docs)}/{len(hwp_files)}개 파일 파싱 완료")

발견된 HWP 파일: 2개

--- 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.hwp ---
'data/국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.hwp' 파일 파싱 성공!
[추출된 텍스트 미리보기]
사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보완 개발
제안요청서

청렴한 업무처리 및 부조리 신고 안내
국민연금공단은 모든 계약업무를 투명하고 공정하게 처리하고 있습니다.
계약과정 및 계약이행 시 금품·향응·편의 제공을 요구하는 직원이 있거나, 지연·학연 등의 사유로 특혜를 제공하는 것을 보거나 들은 경우에는 공단 홈페이지 부조리신고센터로 신고하여 주시기 바랍니다.

부조리
신고센터
국민연금 홈페이지(www.nps.or.kr) 클린센터

레드휘슬(www.redwhistle.org)_[익명성 보장]
2024.3.

사업담당자: 정보지원실 차장 김소옥
전화번호: 063-713-6482

국민연금공단

목차
Ⅰ. 사업 개요
---------------------------
3
1. 사업명
-----------------------------
3
2. 사업기간
-----------------------------
3
3. 사업예산
----------------------------
...

저장 완료: data/국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보_parsed.txt (62366자)

--- 한국수자원공사_용인 첨단 시스템반도체 국가산단 용수공급사업 타당성.hwp ---
'data/한국수자원공사_용인 첨단 시스템반도체 국가산단 용수공급사업 타당성.hwp' 파일 파싱 성공!
[추출된 텍스트 미리보기]
용인 첨단 시스템반도체 국가산단 용수공급사업
타당성 조사 및 기본계획 수립

용역수행 과업지시서
2024. 4

수 도 개 발 처

목 차

1. 일반과업지시서		3
1.1 과 업 명 : 용인 첨단 시스템반도체 국가산단 용수공급사업 타당성조사 및 기본계획 수립 용역		3
1.2 과업의 목